In [1]:
import pandas as pd
from jaal import Jaal

/Users/dralquinta/Documents/DevOps/mati/dep-diag/venv/lib/python3.11/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
sz_file_path = "./dump.csv"
sz_file_csv = pd.read_csv(sz_file_path, usecols=[
    'LocalVMName', 'RemoteVMName', 'ConnectionCount'], header=0)

In [3]:
# Rename columns for edges
df_edges = sz_file_csv.rename(columns={
    "LocalVMName": "from",
    "RemoteVMName": "to",
    "ConnectionCount": "weight"
})

In [4]:
# Ensure 'from' and 'to' columns are strings
df_edges['from'] = df_edges['from'].astype(str)
df_edges['to'] = df_edges['to'].astype(str)

# Group by 'from' and 'to' to aggregate connection counts
df_grouped = df_edges.groupby(['from', 'to'], as_index=False)['weight'].sum()

# Create nodes dataframe from 'from' and 'to' columns
source_nodes = df_edges[['from']].rename(columns={'from': 'id'})
target_nodes = df_edges[['to']].rename(columns={'to': 'id'})
all_nodes = pd.concat([source_nodes, target_nodes]).drop_duplicates().reset_index(drop=True)

# Add additional attributes for nodes
all_nodes['label'] = all_nodes['id']
all_nodes['group'] = 'Group1'  # Example group, you can customize it as needed

# Create a node weight dataframe
node_weights = df_edges.groupby('from', as_index=False)['weight'].sum()
node_weights.columns = ['id', 'weight']

# Merge node weights with the all_nodes dataframe
df_nodes = all_nodes.merge(node_weights, on='id', how='left').fillna(1)

# Customize the edges with tooltips
df_grouped['title'] = df_grouped.apply(
    lambda row: f"Connection Count: {row['weight']}", axis=1)

In [5]:

# Define Jaal visualization options
vis_opts = {
    'height': '600px',
    'physics': {
        'enabled': False
    }
}

# Initialize and plot using Jaal
Jaal(df_grouped, df_nodes).plot(vis_opts=vis_opts)

Parsing the data...Done


No trigger
